In [ ]:
# Preprocessing

import numpy as np
from skimage import color, exposure, transform

NUM_CLASSES = 43
IMG_SIZE = 48

def preprocess_img(img):
    # Histogram normalization in v channel
    hsv = color.rgb2hsv(img)
    hsv[:, :, 2] = exposure.equalize_hist(hsv[:, :, 2])
    img = color.hsv2rgb(hsv)
    
    # central square crop
    min_side = min(img.shape[:-1])
    centre = img.shape[0] // 2, img.shape[1] // 2
    img = img[center[0] - min_side // 2:centre[0] + min_side // 2
              center[1] - min_side // 2:centre[1] + min_side // 2
             :]
    
    # rescale to standard size
    img = transform.resize(img, (IMG_SIZE, IMG_SIZE))
    
    # roll color axis to axis 0
    img = np.rollaxis(img, -1)
    
    return img

# preprocess all training images
# get labels of images from paths
# convert targets to one-hot form as is required by keras

from skimage import io
import os
import glob

def get_class(img_path):
    return int(img_path.split('/')[-2])

root_dir = 'GTSRB/Final_Training/Images/'
imgs = []
labels = []

all_img_paths = glob.glob(os.path.join(root_dir, '*/*.ppm'))
np.random.shuffle(all_img_paths)
for img_path in all_img_paths:
    img = preprocess_img(io.imread(img_path))
    label = get_class(img_path)
    imgs.append(img)
    labels.append(label)
    
X = np.array(imgs, dtype='float32')
# Make one hot targets
Y = np.eye(NUM_CLASSES, dtype='uint8')[labels]


## Models

Use feed forward network with 6 conv layers followed by a fully connected hidden layer

Also use dropout layers in between (dropout regularizes the networks -- prevents overfitting)

All layers have rectified linear unit, relu activations except the output layer

Output layer uses softmax activation -- to output the probability for each of the classes

`Sequential` is a keras container for linear stack of layers

Each of the layers in the model needs to know the input shape it should expect, but it is enough to specify `input_shape` for the first layer of the `Sequential` model.

Rest of the layers do automatic shape inference.

To attach a fully connected layer (aka dense layer) to a convolutional layer, we will have to reshape / flatten the output of the conv layer.  This is achieved by `Flatten` layer

In [ ]:
# Model
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.optimizers import SGD
from keras import backend as K
K.set_image_data_format('channels_first')

def cnn_model():
    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), padding='same',
                    input_shape(3, IMG_SIZE, IMG_SIZE),
                    activation='relu'))
    model.add(Conv2D(32, (3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(64, (3, 3), padding='same',
                    activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(128, (3, 3), padding='same',
                    activation='relu'))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(NUM_CLASSES, activation='softmaxi'))
    return model

Before training the model, we need to configure the model of the learning algorithm and compile it.  We need to specify:

`loss`: loss function we want to optimize.  We cannot use error percentage as it is not continuous and thus not differentiable.  We therefore use a proxy for it: categorical_crossentropy

`optimizer`: we use standard stochastic gradient descent with Nesterov momentum

`metric`: since we are dealing with a classification problem, our metric is accuracy


In [ ]:
model = cnn_model()

# train the model using SGD + momentum
lr = 0.01
sgd = SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
             optimizer=sgd,
             metrics=['accuracy'])
